<a href="https://colab.research.google.com/github/geraw/ProvengoDemo/blob/main/ProvegoTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [46]:
import platform

if platform.system() == "Linux":
    !wget -nc https://downloads.provengo.tech/unix-dist/deb/Provengo-deb.deb
    !apt-get install ./Provengo-deb.deb
else:
    print("If your OS is not Linux, please download and run the installer from https://provengo.tech/download")    

If your OS is not Linux, please download and run the installer from https://provengo.tech/download


In [47]:
%pip install flask

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


# System Under Test

## Setting up Flask app

In [48]:
from flask import Flask, request, jsonify
import threading
import time
import os
import sys
import json
import importlib
import signal


# Create Flask app
app = Flask(__name__)
#run_with_ngrok(app)  # Allows access from outside

# In-memory data store (replace with a database in a real application)
users = []
loans = []
holds = []
books = {}


## A route for resetting the database

In [49]:
# A rute for testing that resets the database and loads initial data
@app.route('/reset', methods=['POST'])
def reset_database():
    global users, loans, holds, books
    
    # Clear all data
    users = []
    loans = []
    holds = []
    books = {}
    
    # If initial data is provided, load it
    if request.is_json:
        data = request.get_json()
        if 'users' in data:
            users.extend(data['users'])
        if 'loans' in data:
            loans.extend(data['loans'])
        if 'holds' in data:
            holds.extend(data['holds'])
        if 'books' in data:
            books.update(data['books'])
    
    return jsonify({
        'message': 'Database reset',
        'status': {
            'users': len(users),
            'loans': len(loans),
            'holds': len(holds),
            'books': len(books)
        }
    }), 200




## User routes

In [50]:
# --- User Routes ---
@app.route('/users', methods=['POST'])
def add_user():
    user = request.get_json()

    if 'id' not in user:
        print("Error: Attempt to add user without id")
        return jsonify({'error': 'user id is required'}), 400
    if  user.get('id') in [u.get('id') for u in users]:
        print("Error: Attempt to add duplicate user")
        return jsonify({'error': 'User already exists'}), 400
    else:
      users.append(user)
      return jsonify({'message': 'User Added', 'user': user}), 201

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    users = [user for user in users if user.get('id') != user_id]
    return jsonify({'message': 'User deleted'}), 200

@app.route('/users', methods=['GET'])
def search_users():
    query = request.args.get('q', '').lower()
    results = [user for user in users if query in str(user).lower()] if query else users
    return jsonify(results)



## Loan routes

In [51]:
# --- Loan Routes ---
@app.route('/loans', methods=['POST'])
def add_loan():
    loan = request.get_json()
    user_id = loan.get('userId')
    book_id = loan.get('bookId')

    # Check that user exists
    if user_id is None or not any(u.get('id') == user_id for u in users):
        return jsonify({'error': 'User does not exist'}), 400

    # Check that book exists
    if book_id is None or str(book_id) not in books:
        return jsonify({'error': 'Book does not exist'}), 400

    loans.append(loan)
    return jsonify({'message': 'Loan added', 'loan': loan}), 201

@app.route('/loans/<int:loan_id>', methods=['DELETE'])
def delete_loan(loan_id):
    global loans
    loans = [loan for loan in loans if loan.get('id') != loan_id]
    return jsonify({'message': 'Loan deleted'}), 200

@app.route('/loans', methods=['GET'])
def search_loans():
    query = request.args.get('q', '').lower()
    results = [loan for loan in loans if query in str(loan).lower()] if query else loans
    return jsonify(results)



## Hold routes

In [52]:
# --- Hold Routes ---
@app.route('/holds', methods=['POST'])
def add_hold():
    hold = request.get_json()
    holds.append(hold)
    return jsonify({'message': 'Hold added', 'hold': hold}), 201

@app.route('/holds/<int:hold_id>', methods=['DELETE'])
def delete_hold(hold_id):
    global holds
    holds = [hold for hold in holds if hold.get('id') != hold_id]
    return jsonify({'message': 'Hold deleted'}), 200

@app.route('/holds', methods=['GET'])
def search_holds():
    query = request.args.get('q', '').lower()
    results = [hold for hold in holds if query in str(hold).lower()] if query else holds
    return jsonify(results)



## Book routes

In [53]:
# --- Book Routes ---
@app.route('/books', methods=['POST'])
def add_book():
    book = request.get_json()

    if 'id' not in book:
        print("Error: Attempt to add book without id")
        return jsonify({'error': 'book id is required'}), 400
    if book.get('id') in books:
        print("Error: Attempt to add duplicate book")
        return jsonify({'error': 'Book already exists'}), 400
    else:
        books[book['id']] = book
        return jsonify({'message': 'Book Added', 'book': book}), 201

@app.route('/books/<book_id>', methods=['DELETE'])
def delete_book(book_id):
    if book_id in books:
        del books[book_id]
        return jsonify({'message': 'Book deleted'}), 200
    return jsonify({'error': 'Book not found'}), 404

@app.route('/books', methods=['GET'])
def search_books():
    query = request.args.get('q', '').lower()
    results = [book for book in books.values() if query in str(book).lower()] if query else list(books.values())
    return jsonify(results)

@app.route('/books/<book_id>', methods=['GET'])
def get_book(book_id):
    if book_id in books:
        return jsonify(books[book_id])
    return jsonify({'error': 'Book not found'}), 404

## Run the server

In [ ]:
import socket
import random

host = socket.gethostbyname(socket.gethostname())
print(f"{host=}")

port = random.randint(1024, 65535)
url = f"http://{host}:{port}"

threading.Thread(target=app.run, kwargs={'host':host,'port':port}).start()


host='172.24.32.1'
 * Serving Flask app '__main__'
 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://172.24.32.1:5917

 * Running on http://172.24.32.1:5917
Press CTRL+C to quitPress CTRL+C to quit

172.24.32.1 - - [22/Apr/2025 12:06:20] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "172.24.32.1 - - [22/Apr/2025 12:06:20] "POST /users HTTP/1.1POST /users HTTP/1.1" 201 -
" 201 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "172.24.32.1 - - [22/Apr/2025 12:06:20] "POST /users HTTP/1.1POST /users HTTP/1.1" 400 -
" 400 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "172.24.32.1 - - [22/Apr/2025 12:06:20] "POST /users HTTP/1.1POST /users HTTP/1.1" 201 -
" 201 -
172.24.32.1 - - [22/Apr/2025 12:06:20] "GET /users HTTP/1.1" 200 -
17

Error: Attempt to add duplicate user


172.24.32.1 - - [22/Apr/2025 12:06:22] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "172.24.32.1 - - [22/Apr/2025 12:06:22] "POST /books HTTP/1.1POST /books HTTP/1.1" 201 -
" 201 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "172.24.32.1 - - [22/Apr/2025 12:06:22] "POST /books HTTP/1.1POST /books HTTP/1.1" 400 -
" 400 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "172.24.32.1 - - [22/Apr/2025 12:06:22] "POST /books HTTP/1.1POST /books HTTP/1.1" 201 -
" 201 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "172.24.32.1 - - [22/Apr/2025 12:06:22] "POST /books HTTP/1.1POST /books HTTP/1.1" 201 -
" 201 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:06:22] "GET /books HT

...

172.24.32.1 - - [22/Apr/2025 12:55:04] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:04] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:55:05] "POST /users HTTP/1.1" 400 -
172.24.32.1 - - [22/Apr/2025 12:55:05] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:05] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:05] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:55:05] "GET /users HTTP/1.1" 200 -


Error: Attempt to add duplicate user


172.24.32.1 - - [22/Apr/2025 12:55:54] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "POST /users HTTP/1.1" 400 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "GET /users HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:55:54] "GET /users HTTP/1.1" 200 -


Error: Attempt to add duplicate user


172.24.32.1 - - [22/Apr/2025 12:56:36] "POST /reset HTTP/1.1" 200 -
172.24.32.1 - - [22/Apr/2025 12:56:36] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:56:36] "POST /users HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:56:36] "POST /books HTTP/1.1" 201 -
172.24.32.1 - - [22/Apr/2025 12:56:36] "POST /loans HTTP/1.1" 400 -


## A reset script that runs before each test

The file `reset.py` is a simple Python script that resets the system under test (SUT) to a known state. We will use Provengoe's `--before` command line argument to run this script before each test.


In [55]:
import platform


# Write the reset.py script
with open("reset.py", "w") as f:
    f.write(f"""\
import requests
test_data = {{
    'users': [{{ 'id': 1, 'name': 'Test User' }}],
    'books': {{'1': {{ 'id': '1', 'title': 'Test Book' }}}}
}}
requests.post('{url}/reset', json=test_data)
""")

# A hack to run reset on linux because the space in "python reset.py" is not allowed
if platform.system() != "Windows":
    !echo '#!/bin/bash' > before_script.sh
    !echo 'python reset.py' >> before_script.sh
    !chmod +x before_script.sh

# Test model

## Creating a new empty model

We first create a new test model called `BookStoreDemo` by running the following command:


In [56]:
!provengo --batch-mode create BookStoreDemo

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:06:19.569 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:06:19.640 ERR  [SETUP] c:\temp\ProvengoDemo\BookStoreDemo Folder already exist. delete the existing folder or create new project with new name.
12:06:19.640 ERR  [Main] Terminating because of previous errors. See logs.
12:06:19.645 ERR  [Main] 40 BadRequest


## Adding file containing general test information

An effective way to set gloabl parameters to the testing model is to use a file that contains general test information. This file can be imported into the test model and used to set global parameters. In this example, we will use a file called `_constatnts.py` to set the global parameters `host` and `port`. This parameters will be available to all our scriptss.

In [57]:
with open("BookStoreDemo/spec/js/_constants.js", "w") as f:
  f.write(f"const {host=}; const {port=};")

## Adding interface function to simplify the test scripts

We will add a file called `interface.py` that contains functions that will be used to simplify the test scripts. This function will be used to make HTTP requests to the SUT. These are just simple wrapper function that hide the unneccesary details of making HTTP requests.

In [ ]:
%%writefile BookStoreDemo//spec//js//interface.js

const svc = new RESTSession("http://" + host + ":" + port, "provengo basedclient", {
    headers: { "Content-Type": "application/json" }
});

// Add a book to the store
function addBook(id, title) {
    svc.post("/books", { body: JSON.stringify({ id: id, title: title, }), });
    bp.log.info("Add book with id " + id + " and title " + title);
}

// Add a user to the store
function addUser(id, name) {
    svc.post("/users", { body: JSON.stringify({ id: id, name: name, }), });
    bp.log.info("Add user with id " + id + " and name " + name);
}

// Try to add a book that already exists
function tryToAddExistingBook(id, title) {
    svc.post("/books", {
        body: JSON.stringify({ id: id, title: title }),
        expectedResponseCodes: [400]
    });
    bp.log.info("Try to add existing book with id " + id + " and title " + title);
}

// Try to add a user that already exists
function tryToAddExistingUser(id, name) {
    svc.post("/users", {
        body: JSON.stringify({ id: id, name: name }),
        expectedResponseCodes: [400]
    });
    bp.log.info("Try to add existing user with id " + id + " and name " + name);
}

// Verify that a user exists
function verifyUserExists(id, name) {
    svc.get("/users", {
        callback: function (response) {
            user = JSON.parse(response.body);
            for (let i = 0; i < user.length; i++) {
                if (user[i].id === id && user[i].name === name) {
                    return pvg.success("User exists");
                }
            }
            return pvg.fail("Expected a user to exists but it does not");
        }
    });
    bp.log.info("Verify user with "+ id + " and name " + name + " exists");
}

// Verify that a user does not exist
function verifyUserDoesNotExist(id, name) {
    svc.get("/users", {
        callback: function (response) {
            user = JSON.parse(response.body);
            for (let i = 0; i < user.length; i++) {
                if (user[i].id === id && user[i].name === name) {
                    return pvg.fail("Expected a user to not exist but it does");
                }
            }
            return pvg.success("User does not exist");
        }
    });
    bp.log.info("Verify user with "+ id + " and " + name + " does not exist");
}


// Verify that a book exists
function verifyBookExists(id, title) {
    svc.get("/books", {
        callback: function (response) {
            book = JSON.parse(response.body);
            for (let i = 0; i < book.length; i++) {
                if (book[i].id === id && book[i].title === title) {
                    return pvg.success("Book exists");
                }
            }
            return pvg.fail("Expected a book to exists but it does not");
        }
    });
    bp.log.info("Verify book with "+ id + " and " + title + " exists");
}

// Verify that a book does not exist
function verifyBookDoesNotExist(id, title) {
    svc.get("/books", {
        callback: function (response) {
            book = JSON.parse(response.body);
            for (let i = 0; i < book.length; i++) {
                if (book[i].id === id && book[i].title === title) {
                    return pvg.fail("Expected a book to not exist but it does");
                }
            }
            return pvg.success("Book does not exist");
        }
    });
    bp.log.info("Verify book with "+ id + " and " + title + " does not exist");
}


// Helper functions for Loan API endpoints
function addLoan(id, userId) {
    svc.post("/loans", { body: JSON.stringify({ id: id, userId: userId }) });
    bp.log.info("Add loan with id " + id + " and userId " + userId);

}

function deleteLoan(id) {
    svc.delete("/loans/" + id);
    bp.log.info("Delete loan with id " + id);
}

function checkLoanExists(id) {
    svc.get("/loans?q=" + id, function(response) {
        const results = JSON.parse(response.body);
        if (results.some(loan => loan.id === id)) {
            pvg.success("Loan " + id + " exists");
        } else {
            pvg.fail("Loan " + id + " not found");
        }
    });
    bp.log.info("Check that loan with id " + id + " exists");
}

function checkLoanDoesNotExist(id) {
    svc.get("/loans?q=" + id, function(response) {
        const results = JSON.parse(response.body);
        if (results.some(loan => loan.id === id)) {
            pvg.fail("Loan " + id + " should not exist but it does");
        } else {
            pvg.success("Loan " + id + " does not exist");
        }
    });
    bp.log.info("Check that loan with id " + id + " does not exist");
}


Overwriting BookStoreDemo//spec//js//interface.js


## Writing our first (linear) test scripts

Our first test script is a standard sequence of HTTP requests that tests the basic functionality of the SUT. The script,  called `hello_world.js`, is shown below:

In [ ]:
%%writefile BookStoreDemo/spec/js/hello-world.js
//@provengo summon rest


bthread("User API", function () {
  addUser(111, "John Doe");
  tryToAddExistingUser(111, "John Doe");
  verifyUserExists(111, "John Doe");
  verifyUserDoesNotExist(222, "Jane Doe");
  addUser(222, "John Doe");
  verifyUserExists(222, "John Doe");
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [115]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:52:35.063 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:52:35.065 INFO [SETUP] Project path: c:\temp\ProvengoDemo\BookStoreDemo
12:52:35.073 INFO [RUN] Preparing to run
12:52:35.305 INFO [RUN>random>Before Test] Running Before Test command: python reset.py
12:52:35.494 INFO [RUN>random>Before Test] Done running Before Test command: python reset.py
12:52:35.672 INFO [RUN>random] B-program started
12:52:35.696 INFO [RUN>random] Selected: [POST {lib:"REST", method:"POST", url:"http://172.24.32.1:5917/users", headers:{Content-Type:"application/json"}, parameters:{}, expectedResponseCodes:[], callback:{}, body

In addition to testing the user-related functionality, it is essential to create test scripts that cover other critical components of the SUT. For example, we can create scripts to test the book-related API endpoints. This ensures that the system behaves as expected when managing books, including adding, verifying, and handling duplicate entries.

By modularizing the test scripts, we can focus on specific functionalities, making it easier to identify and debug issues. Below is an example script that tests the book-related API endpoints:

In [61]:
%%writefile BookStoreDemo/spec/js/hello-world.js
//@provengo summon rest

// Test script for the Book API
bthread("Book API", function () {
    // Add a new book
    addBook(111, "The Great Gatsby");

    // Attempt to add a duplicate book (should fail)
    tryToAddExistingBook(111, "The Great Gatsby");

    // Add more books
    addBook(222, "The Catcher in the Rye");
    addBook(333, "To Kill a Mockingbird");

    // Verify that the books exist
    verifyBookExists(111, "The Great Gatsby");
    verifyBookExists(222, "The Catcher in the Rye");
    verifyBookExists(333, "To Kill a Mockingbird");

    // Verify that a non-existent book does not exist
    verifyBookDoesNotExist(444, "Harry Potter");
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [62]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:06:21.832 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:06:21.834 INFO [SETUP] Project path: c:\temp\ProvengoDemo\BookStoreDemo
12:06:21.844 INFO [RUN] Preparing to run
12:06:22.085 INFO [RUN>random>Before Test] Running Before Test command: python reset.py
12:06:22.284 INFO [RUN>random>Before Test] Done running Before Test command: python reset.py
12:06:22.473 INFO [RUN>random] B-program started
12:06:22.495 INFO [RUN>random] Selected: [addBook {id:111.0, title:"The Great Gatsby", lib:"bp-base"}]
12:06:22.510 INFO [RUN>random] Selected: [POST {lib:"REST", method:"POST", url:"http://172.24.32.1:5917/books"

## Interleaving two scenarios

In [63]:
%%writefile BookStoreDemo/spec/js/hello-world.js
//@provengo summon rest
bthread("Book API", function () {
    addBook(111, "The Great Gatsby"); 
    tryToAddExistingBook(111, "The Great Gatsby");
    addBook(222, "The Catcher in the Rye");
    addBook(333, "The Catcher in the Rye");
    verifyBookExists(111, "The Great Gatsby");
    verifyBookExists(222, "The Catcher in the Rye");
    verifyBookDoesNotExist(444, "Harry Potter");
});


bthread("User API", function () {
  addUser(111, "John Doe");
  tryToAddExistingUser(111, "John Doe");
  verifyUserExists(111, "John Doe");
  verifyUserDoesNotExist(222, "Jane Doe");
  addUser(222, "John Doe");
  verifyUserExists(222, "John Doe");
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [64]:
!provengo analyze -fgv --style=full BookStoreDemo

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:06:23.741 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:06:23.743 INFO [SETUP] Project path: c:\temp\ProvengoDemo\BookStoreDemo
12:06:23.789 INFO [ANALYZE] Max DFS depth: 60 (default value)
12:06:24.784 INFO [ANALYZE] Found 79 states, connected by 120 edges.
12:06:24.860 INFO [ANALYZE] output c:\temp\ProvengoDemo\BookStoreDemo\products\run-source\testSpace.gv

     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\

In [65]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:06:25.211 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:06:25.213 INFO [SETUP] Project path: c:\temp\ProvengoDemo\BookStoreDemo
12:06:25.220 INFO [RUN] Preparing to run
12:06:25.457 INFO [RUN>random>Before Test] Running Before Test command: python reset.py
12:06:25.644 INFO [RUN>random>Before Test] Done running Before Test command: python reset.py
12:06:25.818 INFO [RUN>random] B-program started
12:06:25.844 INFO [RUN>random] Selected: [addBook {id:111.0, title:"The Great Gatsby", lib:"bp-base"}]
12:06:25.860 INFO [RUN>random] Selected: [POST {lib:"REST", method:"POST", url:"http://172.24.32.1:5917/books"

In [119]:
%%writefile BookStoreDemo/spec/js/hello-world.js
//@provengo summon rest



bthread("Loan API", function () {

    // Add users
    addUser(111, "John Doe");
    addUser(222, "Jane Doe");

    // Add books
    addBook(101, "The Great Gatsby");
    //addBook(102, "The Catcher in the Rye");

    // Add two loans
    addLoan(101, 111);
    addLoan(102, 222);

    // Verify that loan 101 exists
    checkLoanExists(101);

    // Delete loan 101
    deleteLoan(101);

    // Verify that loan 101 has been deleted
    checkLoanDoesNotExist(101);
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [120]:
!provengo run BookStoreDemo --before="python reset.py" 

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

12:56:35.829 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
12:56:35.831 INFO [SETUP] Project path: c:\temp\ProvengoDemo\BookStoreDemo
12:56:35.840 INFO [RUN] Preparing to run
12:56:36.075 INFO [RUN>random>Before Test] Running Before Test command: python reset.py
12:56:36.300 INFO [RUN>random>Before Test] Done running Before Test command: python reset.py
12:56:36.488 INFO [RUN>random] B-program started
12:56:36.512 INFO [RUN>random] Selected: [POST {lib:"REST", method:"POST", url:"http://172.24.32.1:5917/users", headers:{Content-Type:"application/json"}, parameters:{}, expectedResponseCodes:[], callback:{}, body